라이브러리 및 데이터불러오기

In [1]:
import pandas as pd
data = pd.read_csv('Korea Income and Welfare.csv')
data

,id,year,wave,region,income,family_member,gender,year_born,education_level,marriage,religion,occupation,company_size,reason_none_worker
0,10101,2005,1,1,614.0,1,2,1936,2,2,2,,,8
1,10101,2011,7,1,896.0,1,2,1936,2,2,2,,,10
2,10101,2012,8,1,1310.0,1,2,1936,2,2,2,,,10
3,10101,2013,9,1,2208.0,1,2,1936,2,2,2,,,1
4,10101,2014,10,1,864.0,1,2,1936,2,2,2,,,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92852,98000701,2014,10,5,11600.0,6,1,1967,5,1,1,874,1,
92853,98000701,2015,11,5,8327.0,6,1,1967,5,1,1,874,1,
92854,98000701,2016,12,5,7931.0,6,1,1967,5,1,1,874,1,
92855,98000701,2017,13,5,8802.0,5,1,1967,5,1,1,874,1,


사용하지 않을 열 제거

In [2]:
data = data.drop(['id','occupation','company_size','reason_none_worker'],axis=1)

In [3]:
year_max = max(data['year'])
year_max

2018

In [4]:
data = data[data['year']==2018]

데이터가 얼마나 줄어들었는지 확인

In [5]:
len(data)

6331

In [6]:
data

,year,wave,region,income,family_member,gender,year_born,education_level,marriage,religion
21,2018,14,1,2330.0,1,2,1945,4,2,1
35,2018,14,1,815.0,1,1,1948,3,2,2
49,2018,14,1,2116.0,1,1,1942,7,3,1
64,2018,14,1,5008.0,5,1,1962,6,1,1
88,2018,14,2,1332.0,1,2,1940,3,2,1
...,...,...,...,...,...,...,...,...,...,...
92817,2018,14,6,557.0,1,2,1947,2,2,2
92825,2018,14,6,1948.0,2,1,1933,3,1,2
92833,2018,14,6,866.0,1,2,1934,3,2,1
92841,2018,14,6,4118.0,2,1,1956,5,1,1


In [7]:
# 중복된 행 삭제
data.drop_duplicates()
len(data) # 중복된 행이 없는 것을 확인

6331

In [8]:
# 누락 데이터 찾기
data.isnull().sum()

year               0
wave               0
region             0
income             0
family_member      0
gender             0
year_born          0
education_level    0
marriage           0
religion           0
dtype: int64

In [13]:
# 통계값 확인하기 
data.describe()

,year,wave,income,family_member,year_born,marriage,religion
count,6331.0,6331.0,6331.000000,6331.000000,6331.000000,6331.000000,6331.000000
mean,2018.0,14.0,4273.601279,2.277365,1955.559627,1.766546,1.521245
std,0.0,0.0,4509.175617,1.240378,16.390786,1.135312,0.499588
min,2018.0,14.0,-46897.000000,1.000000,1922.000000,1.000000,1.000000
25%,2018.0,14.0,1418.500000,1.000000,1942.000000,1.000000,1.000000
50%,2018.0,14.0,2987.000000,2.000000,1954.000000,1.000000,2.000000
75%,2018.0,14.0,5984.500000,3.000000,1969.000000,2.000000,2.000000
max,2018.0,14.0,170204.000000,9.000000,1999.000000,6.000000,2.000000


In [10]:
####### 이부분은 다 계산하고 마지막에 넣어서 어떤 자료인지만 해석하는 용도로 사용할까?
# 값 치환 (보기 편하게 하기 위해서)
data['region'] = data['region'].replace([1,2,3,4,5,6,7],['Seoul','Kyeong-gi','Kyoung-nam','Kyoung-buk','Chung-nam','Gang-won &. Chung-buk', 'Jeolla & Jeju'])
data['education_level'] = data['education_level'].replace([1,2,3,4,5,6,7,8,9],['no education','no education','elementary','middle school','high school','college','university degree','MA','doctoral degree'])
data['gender'] = data['gender'].replace([1,2],['male','female'])


In [11]:
# https://www.kaggle.com/datasets/hongsean/korea-income-and-welfare
# 사이트주소 

In [12]:
#변경된 데이터 확인
data

,year,wave,region,income,family_member,gender,year_born,education_level,marriage,religion
21,2018,14,Seoul,2330.0,1,female,1945,middle school,2,1
35,2018,14,Seoul,815.0,1,male,1948,elementary,2,2
49,2018,14,Seoul,2116.0,1,male,1942,university degree,3,1
64,2018,14,Seoul,5008.0,5,male,1962,college,1,1
88,2018,14,Kyeong-gi,1332.0,1,female,1940,elementary,2,1
...,...,...,...,...,...,...,...,...,...,...
92817,2018,14,Gang-won &. Chung-buk,557.0,1,female,1947,no education,2,2
92825,2018,14,Gang-won &. Chung-buk,1948.0,2,male,1933,elementary,1,2
92833,2018,14,Gang-won &. Chung-buk,866.0,1,female,1934,elementary,2,1
92841,2018,14,Gang-won &. Chung-buk,4118.0,2,male,1956,high school,1,1


In [14]:
# 통계값 확인하기 
data.describe()

,year,wave,income,family_member,year_born,marriage,religion
count,6331.0,6331.0,6331.000000,6331.000000,6331.000000,6331.000000,6331.000000
mean,2018.0,14.0,4273.601279,2.277365,1955.559627,1.766546,1.521245
std,0.0,0.0,4509.175617,1.240378,16.390786,1.135312,0.499588
min,2018.0,14.0,-46897.000000,1.000000,1922.000000,1.000000,1.000000
25%,2018.0,14.0,1418.500000,1.000000,1942.000000,1.000000,1.000000
50%,2018.0,14.0,2987.000000,2.000000,1954.000000,1.000000,2.000000
75%,2018.0,14.0,5984.500000,3.000000,1969.000000,2.000000,2.000000
max,2018.0,14.0,170204.000000,9.000000,1999.000000,6.000000,2.000000


In [16]:
dummy1 = pd.get_dummies(data[['region']])
dummy2 = pd.get_dummies(data[['education_level']])
dummy3 = pd.get_dummies(data[['gender']])

In [20]:
data.corr()

C:\Users\cjf46\AppData\Local\Temp\ipykernel_20720\2627137660.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.corr()


,year,wave,income,family_member,year_born,marriage,religion
year,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wave,NaN,NaN,NaN,NaN,NaN,NaN,NaN
income,NaN,NaN,1.000000,0.546047,0.415581,-0.245391,0.042024
family_member,NaN,NaN,0.546047,1.000000,0.452891,-0.480486,0.072579
year_born,NaN,NaN,0.415581,0.452891,1.000000,0.171251,0.186195
marriage,NaN,NaN,-0.245391,-0.480486,0.171251,1.000000,0.058881
religion,NaN,NaN,0.042024,0.072579,0.186195,0.058881,1.000000
